In [1]:
%load_ext autoreload
%autoreload 2

from celeste_rl.level import *
from celeste_rl.env import *
from celeste_rl.models import *
from rtgym import DEFAULT_CONFIG_DICT
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO

import matplotlib.pyplot as plt
import numpy as np
import cv2
import gymnasium

In [ ]:
# In[10]:

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = CelesteGym
my_config["time_step_duration"] = 0.04
my_config["start_obs_capture"] = 0.04
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 1_000_000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

# run = wandb.init(
#     project="celesterl",
#     config=my_config,
#     sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
#     monitor_gym=True,  # auto-upload the videos of agents playing the game
#     save_code=True,  # optional
# )



env = gymnasium.make("real-time-gym-v1", config=my_config)
wrapenv = Monitor(CustomEnv(env))
obs_space = env.observation_space
act_space = env.action_space

model = PPO("CnnPolicy",
            wrapenv,
            n_steps=2048,
            learning_rate=5e-3,
            policy_kwargs=dict(normalize_images=False),
            verbose=1,
            device='cpu',
            tensorboard_log="./celeste_tensorboard/")


for i in range(10):

    # model.learn(100_000, callback=CustomCallback())
    
    model.learn(100_000)
    
    model.save(f'model_{i}.sav')



/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float64. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: 0.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


Using cpu device
Wrapping the env in a DummyVecEnv.


Logging to ./celeste_tensorboard/PPO_8


/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/rtgym/envs/real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.2643300488125533
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/rtgym/envs/real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1392667822074145
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/rtgym/envs/real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.10949208796955645
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
/home/leopaul/mambaforge/envs/zmq/lib/python3.8/site-packages/rtgym/envs/real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.12433886993676424
  warnings

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.1     |
|    ep_rew_mean     | 0.302    |
| time/              |          |
|    fps             | 21       |
|    iterations      | 1        |
|    time_elapsed    | 95       |
|    total_timesteps | 2048     |
---------------------------------
